## Bibliotecas

In [1]:
import pickle
import statistics
import pandas as pd
from sklearn.metrics import f1_score
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer

In [2]:
# Importar los modelos de clasificación
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier

# importar xgboost
# import xgboost as xgb

## Cargar información

In [3]:
with open("../normalizacion_texto/lista_contenido_normalizado.pkl", "rb") as f:
    corpus = pickle.load(f)

In [4]:
data = pd.read_excel('../apoyo/Rest_Mex_2022.xlsx')
y = data['Polarity'].values

## Train y test

In [5]:
x_train, x_test, y_train, y_test = train_test_split(corpus, y, test_size=0.2, random_state=0)

## Representación de texto

In [6]:
# --> Vector Binario
# vectorizador_binario = CountVectorizer(binary=True, token_pattern=r'(?u)\w\w+|\w\w+\n|\.')

# X_train_vector = vectorizador_binario.fit_transform(x_train)
# X_test_vector = vectorizador_binario.transform(x_test)

# --> Vector frecuencia
# vectorizador_frecuencia = CountVectorizer(token_pattern=r'(?u)\w\w+|\w\w+\n|\.')

# X_train_vector = vectorizador_frecuencia.fit_transform(x_train).toarray()
# X_test_vector = vectorizador_frecuencia.transform(x_test).toarray()

# --> Vector tf-idf
vectorizador_tfidf = TfidfVectorizer(token_pattern=r'(?u)\w\w+|\w\w+\n|\.')

X_train_vector = vectorizador_tfidf.fit_transform(x_train)
X_test_vector = vectorizador_tfidf.transform(x_test)


## KFold Cross Validation

In [21]:
# clf = MultinomialNB()
# clf = LogisticRegression(solver='lbfgs', max_iter=1000)
# clf = RandomForestClassifier(n_estimators=100, max_depth=2, random_state=0)
# clf = KNeighborsClassifier(n_neighbors=3)
clf = MLPClassifier(hidden_layer_sizes=(50, 25), activation='relu', solver='adam', alpha=0.0001, learning_rate='constant', max_iter=200, batch_size='auto', early_stopping=False)
# params = {
#     'objective': 'multi:softmax',  # Problema de clasificación múltiple
#     'num_class': len(set(y_train)),  # Número de clases en el conjunto de datos
#     'max_depth': 3,
#     'eta': 0.1,
# }

skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=0)
f1_macro = []

In [22]:
# --> Iteraciones
for entrenamiento, prueba in skf.split(X_train_vector, y_train):
    # --> Validacion cruzada
    rasgos_entrenamiento = X_train_vector[entrenamiento]
    rasgos_prueba = X_train_vector[prueba]

    clases_entrenamiento = y_train[entrenamiento]
    clases_prueba = y_train[prueba]

    # --> Entrenamiento
    clf.fit(rasgos_entrenamiento, clases_entrenamiento)
    clase_predicha = clf.predict(rasgos_prueba)

    # --> Métrica de rendimiento
    f1_macro.append(f1_score(clases_prueba, clase_predicha, average='macro'))

    # --> Porcentaje avanzado
    print(f"\rProgreso: {len(f1_macro)/skf.get_n_splits():.1%}", end="")

Progreso: 100.0%

In [23]:
# --> Resultados
print(f"F1 macro: {round(statistics.mean(f1_macro), 2)*100}%")

# --> Guardar resultados en txt uno sobre otro
with open("resultados.txt", "a") as f:
    f.write(f"MLPClassifier: {round(statistics.mean(f1_macro), 2)*100}%\n")

F1 macro: 44.0%
